## Scraping Fed information from MarketWatch

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from collections import defaultdict
import re

chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

#### Selenium cheat sheet

In [2]:
#x = driver.find_element_by_name('name')
#x.send_keys('type a string' or Keys.RETURN)

#button = driver.find_element_by_xpath('//tag[text()="TEXT"]')
#button.click()

#button2 = driver.find_element_by_xpath('//tag[contains(@class_style_or_other_tag, "=STRING")]')
#button2.click()
#sometimes has /b at the end

In [3]:
url = 'http://www.marketwatch.com'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [9]:
#Search 'yellen'
path = '//a[@class="btn btn--outline btn--search"]'
driver.find_element_by_xpath(path).click()
search = driver.find_element_by_class_name('search__input')
search.send_keys('yellen')
search.send_keys(Keys.RETURN)

#Advanced Searcb
driver.find_element_by_id('refinesearchtoggle').click()
driver.find_element_by_xpath('//input[@type="checkbox"]').click()

#Set Subject
search_mode = '//select[@id="m"]/option[@value="Subject"]'
driver.find_element_by_xpath(search_mode).click()

#Results Per Page
results = '//select[@id="rpp"]/option[@value="50"]'
driver.find_element_by_xpath(results).click()

#Economic News
set_mode = '//select[@id="mp"]/option[@value="ECAT"]'
driver.find_element_by_xpath(set_mode).click()

#Set date
date = driver.find_element_by_id('bdv')
date.click()
date.send_keys('09/29/2017')

#Search
driver.find_element_by_xpath('//input[@value="Search"]').click()
current_url = driver.current_url

In [10]:
soup = BeautifulSoup(requests.get(current_url).text, 'html.parser')

In [11]:
links = dict()
pages = soup.find_all(class_='searchresult')
for div in pages:
    attrs = dict()
    attrs['url'] = div.a['href']
    attrs['date'] = div.next_sibling.span.text
    links[div.a.text] = attrs

In [12]:
links

{'2-year Treasury yield is highest since 2008': {'date': '4:49 p.m. Sept. 21, 2017',
  'url': 'http://www.marketwatch.com/story/2-year-treasury-yield-holds-around-highest-level-since-2008-2017-09-21'},
 '2-year Treasury yield records longest string of gains in 2 months': {'date': '4:31 p.m. Sept. 14, 2017',
  'url': 'http://www.marketwatch.com/story/treasury-yields-tick-higher-as-consumer-inflation-data-looms-2017-09-14'},
 '5 things to know about Kevin Warsh, who may be the next Fed chairman': {'date': '2:28 p.m. Sept. 29, 2017',
  'url': 'http://www.marketwatch.com/story/five-things-to-know-about-kevin-warsh-who-may-be-the-next-fed-chairman-2017-09-29'},
 'A sexist view of marriage and why the world is running out of antibiotics': {'date': '5:25 p.m. Sept. 20, 2017',
  'url': 'http://www.marketwatch.com/story/a-sexist-view-of-marriage-and-why-the-world-is-running-out-of-antibiotics-2017-09-20'},
 'Dollar holds on to gains after Yellen reiterates case for rate hikes': {'date': '4:12 p

In [13]:
def search_links(links):
    for name, attrs in links.items():
        driver = webdriver.Chrome(chromedriver)
        driver.get(attrs['url'])
        soup = BeautifulSoup(requests.get(attrs['url']).text, 'html.parser')
        paragraphs = soup.find(id='article-body').find_all('p')
        attrs.update({'content': [' '.join(t.split()) for t in [''.join(p.find_all(text=True)) for p in paragraphs if bool(p.findChildren('strong')) == False]]})
        driver.close
    return links
articles = search_links(links)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [16]:
articles

{'2-year Treasury yield is highest since 2008': {'content': ['U.S. Treasury prices were little changed Thursday, leaving the 2-year Treasury yield near an almost nine-year high one day after the Federal Reserve indicated that it still plans to deliver another rate increase in 2017.',
   'The U.S. central bank kept its benchmark interest rate unchanged between 1% to 1.25%, but said it would begin its historic unwind of a more than $4.5 trillion balance sheet in October, as expected.',
   'The 2-year Treasury note yield TMUBMUSD02Y, +1.94% the most sensitive to Fed policy shifts, was little changed at 1.442%, after hitting its highest level since Nov. 3, 2008, according to WSJ Market Data Group. The 10-year Treasury note yield TMUBMUSD10Y, +1.21% was steady at 2.278%, holding on to its highest level since Aug. 8. The 30-year bond yield TMUBMUSD30Y, -0.21% traded 1.3 basis point lower at 2.808%.',
   'Bond prices move in the opposite direction of yields.',
   'So far this month, the 2-yea

In [18]:
for name, attrs in articles.items():
    attrs['content'] = ''.join(attrs['content'])

KeyError: 'content'

In [19]:
for name, attrs in articles.items():
    attrs['content'] = re.sub(r'\.([A-Z])', r'. \1', attrs['content'])

NameError: name 'articles' is not defined

In [ ]:
for a in articles.keys():
    print(a)

In [ ]:
#econ_politics_selector = '//a[contains(text(), "Economy & Politics")]'
#driver.find_element_by_xpath(econ_politics_selector).click()

In [ ]:
#the_fed = '//a[@href="/federal-reserve"]'
#driver.find_element_by_xpath(the_fed).click()